In [1]:
import torchvision
torchvision.disable_beta_transforms_warning()
from torchvision.transforms import v2
from torchvision import datasets, transforms, models

import timm
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
import cv2
import numpy as np
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error

import zipfile
import fnmatch
from PIL import Image

#For texture extraction
from skimage import io, feature

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

c:\CS4487\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#testing features, not actual -> change to real later
def extract_texture_features(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray_image = gray_image.astype(np.uint8)  # Ensure the image is 8-bit integer
    radius = 3
    n_points = 8 * radius
    lbp = feature.local_binary_pattern(gray_image, n_points, radius, method='uniform')
    (hist, bin) = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)  # Normalize the histogram
    return hist

def extract_color_features(image):
    hist_b = cv2.calcHist([image], [0], None, [256], [0, 256])
    hist_g = cv2.calcHist([image], [1], None, [256], [0, 256])
    hist_r = cv2.calcHist([image], [2], None, [256], [0, 256])
    return np.stack([hist_b, hist_g, hist_r], axis=0)

def extract_shape_features(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray_image = gray_image.astype(np.uint8)  # Ensure the image is 8-bit
    edges = cv2.Canny(gray_image, 100, 200)
    return edges[np.newaxis, :, :]  # Add an extra dimension for compatibility



In [3]:
def load_image_from_zip(zip_path, img_path):
    with zipfile.ZipFile(zip_path, 'r') as zf:
        with zf.open(img_path) as file:
            img = Image.open(file)
            return img.convert("RGB")  # Ensure the image is in RGB format
        
class ZipImageFolderDataset(datasets.ImageFolder):
    def __init__(self, zip_path, root, transform=None):
        self.zip_path = zip_path
        self.root = root
        self.transform = transform
        self.classes = ['0_real', '1_fake']
        self.img_paths = self._get_image_paths()

    def _get_image_paths(self):
        img_paths = []
        with zipfile.ZipFile(self.zip_path, 'r') as zf:
            for file_info in zf.infolist():
                name = file_info.filename
                if fnmatch.fnmatch(name, f"{self.root}/*.jpg"):
                    label = 0 if '0_real' in name.split('/')[1] else 1
                    img_paths.append((name, label))
        return img_paths

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, index):
        img_path, label = self.img_paths[index]
        img = load_image_from_zip(self.zip_path, img_path)
        if self.transform:
            img = self.transform(img)
        
        # Ensure the image is now a tensor
        if not isinstance(img, torch.Tensor):
            raise TypeError(f"Expected image to be a tensor, but got {type(img)}.")
        
        # Convert tensor to numpy array for feature extraction
        img_np = img.numpy().transpose(1, 2, 0)
        
        # Extract features
        texture_features = extract_texture_features(img_np).flatten()
        color_features = extract_color_features(cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)).flatten()
        shape_features = extract_shape_features(img_np).flatten()
        
        # Concatenate features into a single array
        features = np.concatenate([texture_features, color_features, shape_features], axis=0)
        features = torch.tensor(features).float()
        
        return img, features, label

In [ ]:
def evaluate(model, val_loader):
    # Validation phase
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            max, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Validation Accuracy: {accuracy}')

In [5]:
def save_model(model, path='testing.pth'):
    torch.save(model.state_dict(), path)
    for name, param in model.state_dict().items():
        with open(r"weights.txt",'a') as file:
            file.write(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")  
    with open(r"weights.txt",'a') as file:
        file.write('-'*100 + '\n') 
    print("Model saved successfully!")

In [6]:
def load_model(path='testing.pth'):
    model = timm.create_model('seresnext101_32x4d', pretrained=True)
    model.load_state_dict(torch.load(path, map_location=DEVICE))
    print("Model loaded successfully!")
    return model

In [7]:
def train_model(model, train_loader, optimizer, criterion, epochs):
    model.train()
    for epoch in range(epochs):
        running_loss = 0
        print(f"Epoch {epoch+1} started...")
        print(f"length of train_loader: {len(train_loader)}")
        start_time = time.time()
        
        batch = 1
        for inputs, features, labels in train_loader:
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            features = features.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            print(f'Batch {batch} completed...')
            batch += 1
            

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Time: {time.time()-start_time:.2f}s')
        save_model(model, path=f'testing_{epoch}.pth')

In [8]:
def load_data(zip_path, batch_size, image_size):
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    train_dir = "AIGC-Detection-Dataset/train"
    val_dir = "AIGC-Detection-Dataset/val"
    # test_dir = "AIGC-Detection-Dataset/val"

    train_dataset = ZipImageFolderDataset(zip_path, train_dir, transform=transform)
    val_dataset = ZipImageFolderDataset(zip_path, val_dir, transform=transform)
    # test_dataset = ZipImageFolderDataset(zip_path, test_dir, transform=transform)
    print(f"Data prepared:\nTrain: {len(train_dataset)}, Val: {len(val_dataset)}")

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
    # test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
    
    print("Data loaded")
    return train_loader, val_loader

In [9]:
model = timm.create_model('seresnext101_32x4d', pretrained=True)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# Modify the first convolutional layer
original_conv1 = model.conv1
new_conv1 = nn.Conv2d(3, original_conv1.out_channels, kernel_size=original_conv1.kernel_size,
                      stride=original_conv1.stride, padding=original_conv1.padding, bias=False)
with torch.no_grad():
    new_conv1.weight[:, :3, :, :] = original_conv1.weight[:, :3, :, :]
model.conv1 = new_conv1

# Load the data
zip_path = '..\AIGC-Detection-Dataset.zip'
batch_size = 32
image_size = 512
train_loader, val_loader = load_data(zip_path, batch_size, image_size)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Adjust the final layer for binary classification
model.fc = nn.Linear(model.fc.in_features, 2)

model = model.to(DEVICE)
train_model(model, train_loader, optimizer, criterion, 5)
save_model(model, 'seresnext_finetuned.pth')
evaluate(model, val_loader)


Data prepared:
Train: 45000, Val: 5000
Data loaded
Epoch 1 started...
length of train_loader: 1406


KeyboardInterrupt: 